## CPU sharing

* Use same approach OS uses for sharing CPU between processes — Limited Direct Execution (LDE)
* _Recall: How does LDE work in OS?_
    * Put CPU in user mode
    * Give process exclusive access to CPU
    * Process issues a syscall to perform privileged operation
    * Timer interrupt allows OS to regain control
* To use LDE with a VM
    * Put CPU in user mode
    * Give VM exclusive access to CPU
    * Process issues a syscall -OR- OS attempts to perform privileged operation
    * Timer interrupt allows hypervisor to regain control
* Recall: _How does a process make a syscall?_ — load syscall number into register `x8` and execute `svc` instruction
* _What happens when a process makes a syscall?_
    1. CPU switches to kernel mode
    2. CPU saves registers to the OS stack
    3. CPU uses system call number to index into OS's table of syscall handlers 
    4. Branch and link to syscall handler code
    5. Restore registers from the OS stack
    6. Switch CPU to user mode
    7. Resume execution after `svc` 
* Syscalls with a type 1 hypervisor
    * Processes in virtualized OS do the same as before
    * Hypervisor’s syscall handler is invoked by hardware instead of OS’s syscall handler
    * Actually want to invoke OS's syscall handler — the hypervisor does this
    * Hypervisor knows where OS trap table is located, because populating syscall table is a privileged instruction, so hypervisor interposed on these operations when OS booted
* Steps when process in virtualized OS makes a syscall
    1. CPU switches to kernel mode
    2. CPU saves registers to the hypervisor stack
    3. CPU uses system call number to index into VM's table of syscall handlers 
    4. **Switch CPU from kernel to user mode**
    5. Branch and link to syscall handler code
    6. **Switch CPU from user to kernel mode**
    7. Restore registers from the hypervisor stack
    8. Switch CPU to user mode
    9. Resume execution after `svc`
* Extra work to always trap to hypervisor, which then invokes OS code
* Possible solutions
    * Hardware support — e.g., supervisor mode that is between user and kernel mode
    * Para-virtualization — modify the OS so it's aware there is a hypervisor and the two can interact directly
    * Containers — use a single OS, but create groups of processes that are isolated from other processes
        * OS's process abstraction already isolates each process's CPU time, memory, and registers, but processes can still use syscalls to pass messages to each other, wait for each other, access a common set of files, etc.